# Emotional anaysis of tweets (Subset)

In [1]:
import pandas as pd
import numpy as np
import torch
import matplotlib.pyplot as plt
import transformers
import torch
import seaborn as sns

In [2]:
df = pd.read_csv('/notebooks/evanc/KoReCOVID/resources/Pol5_Clean.csv').sample(5000)

In [3]:
df.head()

,Unnamed: 0,time,id,text,user,uid,mention_username,mention_id,mention_realname,url,...,user_listed_count,user_description,user_geo_enabled,user_verified,lang,rtb,mentioncheck,replycheck,month,week
146590,146590,2020-05-21 23:35:37+00:00,1263614456829022209,@dcexaminer @JoeBiden @realDonaldTrump This he...,ArmedInfidel_US,1472543887,dcexaminer JoeBiden realDonaldTrump realDonald...,18956073 939091 25073877 25073877,Washington Examiner Joe Biden Donald J. Trump ...,https://t.co/WmOTDfqLDF,...,4,"WARNING: If you are easily offended, you bette...",False,False,en,True,True,False,5,21
60869,60869,2020-05-17 17:57:50+00:00,1262079896785784834,@realDonaldTrump people are in a desperate sit...,MikeSireli,755950504627273728,realDonaldTrump,25073877,Donald J. Trump,https://t.co/OQmgBPrNQT,...,0,NaN,False,False,en,True,True,True,5,20
109100,109100,2020-05-20 15:40:54+00:00,1263132600480206849,@tedlieu @IAmJohnAles Ol Ted out here spitting...,Benjay_mann,911645740941942785,tedlieu IAmJohnAles,21059255 140586054,Ted Lieu John Ales*,https://t.co/OB8pEr7ZhB,...,0,"Sports, video games, fishing and a family man....",False,False,en,True,True,True,5,21
309242,309242,2020-05-31 07:17:07+00:00,1266992084638105601,@realDonaldTrump You are a pathetic excuse for...,AppalachianSpi2,1263887973625536519,realDonaldTrump,25073877,Donald J. Trump,https://t.co/GtBBHG5ax4,...,0,NaN,False,False,en,True,True,True,5,22
245739,245739,2020-05-26 06:33:24+00:00,1265169143835299849,@SexCounseling @brithume @realDonaldTrump Peop...,rustyisleaving,3383186015,SexCounseling brithume realDonaldTrump,204444440 112047805 25073877,"Dawn Michael, PhD💗 Brit Hume Donald J. Trump",NaN,...,24,NaN,True,False,en,True,True,False,5,22


In [4]:
df.shape

(5000, 31)

In [5]:
df = df.drop('Unnamed: 0', axis=1)

In [6]:
from transformers import pipeline

2022-04-16 20:31:26.501853: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-16 20:31:26.501917: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [7]:
# First we will classify based on specific emotions
classifier = pipeline('text-classification', model='bhadresh-savani/distilbert-base-uncased-emotion', return_all_scores=True)

In [8]:
df['Emotions'] = df['text'].apply(lambda tweet: classifier(tweet))

In [10]:
#Now we are going to find sentiments with bertweet
SentClassify = pipeline('text-classification', model='finiteautomata/bertweet-base-sentiment-analysis', return_all_scores=True)

In [12]:
#This may be less accurate as this is truncated to the old twitter length
df['Bert_Scents'] = df['text'].apply(lambda tweet: SentClassify(tweet[:127]))

In [13]:
#Next we will use VADER
import nltk

In [14]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [15]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [17]:
df['VADER_Scents'] = df['text'].apply(lambda tweet: sid.polarity_scores(tweet))

In [18]:
df.to_csv('/notebooks/evanc/KoReCOVID/resources/Pol5_Clean_Sample_Scores.csv')

In [21]:
sns.barplot(data=df, 

Help on function barplot in module seaborn.categorical:

barplot(*, x=None, y=None, hue=None, data=None, order=None, hue_order=None, estimator=<function mean at 0x7f54100fd9d0>, ci=95, n_boot=1000, units=None, seed=None, orient=None, color=None, palette=None, saturation=0.75, errcolor='.26', errwidth=None, capsize=None, dodge=True, ax=None, **kwargs)
    Show point estimates and confidence intervals as rectangular bars.
    
    A bar plot represents an estimate of central tendency for a numeric
    variable with the height of each rectangle and provides some indication of
    the uncertainty around that estimate using error bars. Bar plots include 0
    in the quantitative axis range, and they are a good choice when 0 is a
    meaningful value for the quantitative variable, and you want to make
    comparisons against it.
    
    For datasets where 0 is not a meaningful value, a point plot will allow you
    to focus on differences between levels of one or more categorical
    variab